In [ ]:
import copy, random, os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from tqdm import tqdm

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [ ]:
# ================= Dataset Selector ==================
dataset_name = "CIFAR10"   # Change this line: options = ["CIFAR10", "FEMNIST", "EMNIST"]

def get_cifar10():
    # your existing CIFAR10 loader code
    return train_dataset, test_dataset

def get_femnist():
    # FEMNIST loader function
    return train_dataset, test_dataset

def get_emnist():
    # EMNIST loader function
    return train_dataset, test_dataset

dataset_loaders = {
    "CIFAR10": get_cifar10,
    "FEMNIST": get_femnist,
    "EMNIST": get_emnist,
}

train_dataset, test_dataset = dataset_loaders[dataset_name]()


In [ ]:
# Federated parameters
num_clients = 5
rounds = 200                # increase for better accuracy
local_epochs = 50
batch_size = 128
lr = 0.001

# Noise injection parameters
alphas = [0.5, 0.3, 0.2]
inject_fractions = [0.3, 0.2, 0.1]
big_change_threshold = 0.05


In [ ]:
# Transform + normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

# Download dataset
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_set  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

print(f"Train samples: {len(train_set)}, Test samples: {len(test_set)}")


100%|██████████| 170M/170M [00:06<00:00, 26.8MB/s]


Train samples: 50000, Test samples: 10000


In [ ]:
total = len(train_set)
base_len = total // num_clients
lengths = [base_len] * num_clients
lengths[-1] += total - sum(lengths)  # remainder to last

client_datasets = random_split(train_set, lengths)

for i, ds in enumerate(client_datasets):
    print(f"Client {i+1}: {len(ds)} samples")

test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)


Client 1: 10000 samples
Client 2: 10000 samples
Client 3: 10000 samples
Client 4: 10000 samples
Client 5: 10000 samples


In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(128*4*4, 256)  # CIFAR-10: 32->16->8->4
        self.fc2 = nn.Linear(256, 10)       # 10 classes

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)


In [ ]:
def train_local(model, dataset, epochs=50, batch_size=64, lr=0.001, device=device):
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    for _ in range(epochs):
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = F.cross_entropy(outputs, labels)
            loss.backward()
            optimizer.step()
    # return state dict on CPU
    state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
    return state


In [ ]:
def fedavg(client_weights):
    new_state = copy.deepcopy(client_weights[0])
    for k in new_state.keys():
        for i in range(1, len(client_weights)):
            new_state[k] = new_state[k] + client_weights[i][k]
        new_state[k] = new_state[k] / len(client_weights)
    return new_state


In [ ]:
def inject_noise(weights, alpha, inject_fraction):
    noisy = copy.deepcopy(weights)
    for key, val in noisy.items():
        if not torch.is_floating_point(val):
            continue
        flat = val.view(-1)
        n = int(len(flat) * inject_fraction)
        if n <= 0:
            continue
        idx = random.sample(range(len(flat)), n)
        noise = alpha * torch.randn(n)
        flat[idx] += noise
        noisy[key] = flat.view(val.shape)
    return noisy


In [ ]:
def evaluate_with_weights(weights_dict, loader, device=device):
    model = SimpleCNN().to(device)
    state = {k: v.to(device) for k, v in weights_dict.items()}
    model.load_state_dict(state)
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            preds = model(imgs).argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return 100.0 * correct / total


In [ ]:
global_model = SimpleCNN().to(device)
prev_global_weights = {k: v.cpu().clone() for k, v in global_model.state_dict().items()}
round_accuracies = []

for r in range(rounds):
    print(f"\n--- Round {r+1}/{rounds} ---")
    client_weights = []
    for c in range(num_clients):
        local_model = SimpleCNN()
        local_model.load_state_dict({k: v.clone() for k,v in global_model.state_dict().items()})
        local_state = train_local(local_model, client_datasets[c], epochs=local_epochs, batch_size=batch_size, lr=lr, device=device)
        client_weights.append(local_state)
    # aggregate
    global_state = fedavg(client_weights)
    global_model.load_state_dict({k: v.to(device) for k,v in global_state.items()})
    acc = evaluate_with_weights(global_state, test_loader, device=device)
    round_accuracies.append({"round": r+1, "accuracy": acc})
    print(f"Round {r+1}: Test Accuracy = {acc:.2f}%")

final_global_weights = {k: v.cpu().clone() for k,v in global_model.state_dict().items()}
print(f"\nFinal Test Accuracy: {round_accuracies[-1]['accuracy']:.2f}%")



--- Round 1/10 ---
Round 1: Test Accuracy = 55.10%

--- Round 2/10 ---
Round 2: Test Accuracy = 67.17%

--- Round 3/10 ---
Round 3: Test Accuracy = 71.37%

--- Round 4/10 ---
Round 4: Test Accuracy = 73.39%

--- Round 5/10 ---
Round 5: Test Accuracy = 74.38%

--- Round 6/10 ---
Round 6: Test Accuracy = 74.42%

--- Round 7/10 ---
Round 7: Test Accuracy = 74.60%

--- Round 8/10 ---
Round 8: Test Accuracy = 75.01%

--- Round 9/10 ---
Round 9: Test Accuracy = 75.07%

--- Round 10/10 ---
Round 10: Test Accuracy = 74.64%

Final Test Accuracy: 74.64%


In [ ]:
def compare_weights(prev, curr, noisy, alpha, frac, acc_noisy, threshold=big_change_threshold):
    rows = []
    sn = 1
    for name, prev_w in prev.items():
        curr_w, noisy_w = curr[name], noisy[name]
        if not torch.is_floating_point(prev_w):
            continue
        diff = (noisy_w - curr_w).view(-1)
        delta_mean = diff.abs().mean().item()
        big_changes = int((diff.abs() >= threshold).sum().item())
        rows.append({
            "S.No": sn,
            "Layer Name": name,
            "Δ_mean": round(delta_mean, 8),
            "Big_changes(>=%.2f)"%threshold: big_changes,
            "Alpha": alpha,
            "Inject Fraction": frac,
            "Accuracy After Noisy Load (%)": round(acc_noisy, 4)
        })
        sn += 1
    return rows

all_rows = []
for alpha in alphas:
    for frac in inject_fractions:
        noisy = inject_noise(final_global_weights, alpha, frac)
        acc_noisy = evaluate_with_weights(noisy, test_loader, device=device)
        rows = compare_weights(prev_global_weights, final_global_weights, noisy, alpha, frac, acc_noisy)
        all_rows.extend(rows)
        print(f"Alpha {alpha}, frac {frac}: accuracy after noisy load = {acc_noisy:.4f}%")

df_layers = pd.DataFrame(all_rows)
df_rounds = pd.DataFrame(round_accuracies)

df_layers.to_csv("fedavg_cifar10_layerwise_noise_accuracy.csv", index=False)
df_rounds.to_csv("fedavg_cifar10_roundwise_accuracy.csv", index=False)
print("CSV files saved.")


NameError: name 'big_change_threshold' is not defined

In [ ]:
from google.colab import files

# Download the generated CSVs
files.download("fedavg_cifar10_layerwise_noise_accuracy.csv")
files.download("fedavg_cifar10_roundwise_accuracy.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_layers = pd.DataFrame(all_rows)
df_layers.to_csv("fedavg_cifar10_layerwise_noise_accuracy.csv", index=False)


In [ ]:
from google.colab import files
files.download("fedavg_cifar10_layerwise_noise_accuracy.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

df_layers = pd.read_csv("fedavg_cifar10_layerwise_noise_accuracy.csv")
df_layers.head(20)   # show first 20 rows


,S.No,Layer Name,Δ_mean,Big_changes(>=0.05),Alpha,Inject Fraction,Accuracy After Noisy Load (%)
0,1,conv1.weight,0.124295,243,0.5,0.3,9.24
1,2,conv1.bias,0.133942,9,0.5,0.3,9.24
2,3,conv2.weight,0.120238,5102,0.5,0.3,9.24
3,4,conv2.bias,0.106183,19,0.5,0.3,9.24
4,5,conv3.weight,0.119509,20367,0.5,0.3,9.24
5,6,conv3.bias,0.113708,35,0.5,0.3,9.24
6,7,fc1.weight,0.119586,144756,0.5,0.3,9.24
7,8,fc1.bias,0.122951,68,0.5,0.3,9.24
8,9,fc2.weight,0.123270,692,0.5,0.3,9.24
9,10,fc2.bias,0.096343,3,0.5,0.3,9.24


In [ ]:
import os
print(os.getcwd())
!ls -lh


/content
total 16K
drwxr-xr-x 3 root root 4.0K Sep 12 13:15 data
-rw-r--r-- 1 root root 3.8K Sep 12 15:21 fedavg_cifar10_layerwise_noise_accuracy.csv
-rw-r--r-- 1 root root   94 Sep 12 14:59 fedavg_cifar10_roundwise_accuracy.csv
drwxr-xr-x 1 root root 4.0K Sep  9 13:46 sample_data


In [ ]:
import pandas as pd
import torch

def compare_weights_simplified(prev, curr, noisy, alpha, frac, acc_noisy,
                               threshold=0.05, top_k=5):
    rows = []
    sn = 1
    for name, prev_w in prev.items():
        curr_w, noisy_w = curr[name], noisy[name]
        if not torch.is_floating_point(prev_w):
            continue

        # Flatten and calculate differences
        diff = (noisy_w - curr_w).view(-1)
        delta_mean = diff.abs().mean().item()
        big_changes = int((diff.abs() >= threshold).sum().item())

        rows.append({
            "S.No": sn,
            "Layer Name": name,
            "Prev Weight (3)": prev_w.view(-1)[:3].cpu().numpy().round(6).tolist(),
            "Curr Weight (3)": curr_w.view(-1)[:3].cpu().numpy().round(6).tolist(),
            "Noisy Weight (3)": noisy_w.view(-1)[:3].cpu().numpy().round(6).tolist(),
            "Δ_mean": round(delta_mean, 6),
            "Big_changes (≥0.05)": big_changes,
            "Alpha": alpha,
            "Inject Fraction": frac,
            "Accuracy After Noise (%)": acc_noisy
        })
        sn += 1

    # Convert to DataFrame
    df = pd.DataFrame(rows)

    # Pick top-k layers by Δ_mean (most affected)
    df_sorted = df.sort_values(by="Δ_mean", ascending=False).head(top_k).reset_index(drop=True)
    return df_sorted


In [ ]:
# Example: capture previous, current, and noisy weights
prev = {k: v.clone() for k, v in model.state_dict().items()}  # previous weights

# After training step, update current weights
curr = {k: v.clone() for k, v in model.state_dict().items()}

# After injecting noise, capture noisy weights
noisy_model = inject_noise(model, alpha=0.5, frac=0.3)  # <-- your noise function
noisy = {k: v.clone() for k, v in noisy_model.state_dict().items()}


NameError: name 'model' is not defined

In [ ]:
alpha = 0.5
frac = 0.3
acc_noisy = 9.24   # or wherever you log accuracy after noisy load

df_simplified = compare_weights_simplified(prev, curr, noisy, alpha, frac, acc_noisy, top_k=5)


NameError: name 'prev' is not defined

In [ ]:
from google.colab import files
files.download("/content/fedavg_cifar10_layerwise_noise_accuracy.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>